In [1]:

import scanpy as sc
import os
import pandas as pd
from scvi.model.utils import mde

In [2]:
input_file = '/mnt/sata3/Dhruv_Analysis_1/2025_05_22_IL-15_KO'
adata = sc.read(os.path.join(input_file, "combined", '07_neighborhoods.h5ad'))

/home/dpatravali/miniforge3/envs/scvi_env_gpu/lib/python3.9/site-packages/anndata/_core/anndata.py:1818: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [3]:
sc.pp.normalize_total(adata, target_sum=1e6)
sc.pp.log1p(adata)
sc.tl.rank_genes_groups(adata, groupby = 'leiden', method = 'wilcoxon')

KeyError: 'leiden'

In [4]:
sc.pl.rank_genes_groups(adata, n_genes=20)

KeyError: 'names'

getting top cluster genes for chatgpt

In [5]:

# Extract the ranking results
ranking = adata.uns['rank_genes_groups']

# Get the group names (clusters)
groups = ranking['names'].dtype.names

# Create an empty dictionary to store the top 15 genes for each group
top_genes_per_group = {}

# Iterate over each group to extract the top 15 genes
for group in groups:
    top_genes_per_group[group] = ranking['names'][group][:15]

# Convert the dictionary to a pandas DataFrame for easy viewing
top_genes_df = pd.DataFrame(top_genes_per_group)

top_genes_df.to_csv('top_genes_20.csv')

KeyError: 'names'

In [6]:
cluster_identities = {
    0: 'Proliferating Cells',
    1: 'Osteoclasts',
    2: 'Fibroblasts',
    3: 'CD4 T Cells',
    4: 'Endothelial Cells',
    5: 'Macrophages',
    6: 'Dendritic Cells',
    7: 'Monocytes',
    8: 'Macrophages',
    9: 'Endothelial Cells',
    10: 'Dendritic Cells',
    11: 'CD8 T Cells',
    12: 'Pericytes',
    13: 'Smooth Muscle Cells',
    14: 'Tumor Cells',
    15: 'Macrophages',
    16: 'Macrophages',
    17: 'Dendritic Cells',
    18: 'Macrophages',
    19: 'Macrophages',
    20: 'T Cells',
    21: 'Erythrocytes',
    22: 'Mesothelial Cells',
    23: 'B Cells', #Needs subclustering
    24: 'Endothelial Cells',
    25: 'Myofibroblasts',
    26: 'Fibroblasts'
}


In [7]:
adata.obs['cell_types'] = adata.obs['leiden'].astype(int).map(cluster_identities)

KeyError: 'leiden'

In [8]:
adata.write(os.path.join(input_file, "combined", 'cell_types_assigned.h5ad'))

In [9]:
adata = sc.read(os.path.join(input_file, "combined", 'cell_types_assigned.h5ad'))

/home/dpatravali/miniforge3/envs/scvi_env_gpu/lib/python3.9/site-packages/anndata/_core/anndata.py:1818: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [ ]:
sc.set_figure_params(dpi=400)
sc.pl.embedding(adata, basis='mde', color=['cell_types', 'leiden'], legend_loc= 'on data')

subclustering CD4 T cells

In [13]:
tcells = adata[adata.obs['cell_types'] == 'CD4 T Cells'].copy()

sc.tl.pca(tcells)
sc.pp.neighbors(tcells)
sc.tl.leiden(tcells)

In [ ]:
sc.tl.rank_genes_groups(tcells, groupby = 'leiden', method = 'wilcoxon')

In [15]:
# Extract the ranking results
ranking = tcells.uns['rank_genes_groups']

# Get the group names (clusters)
groups = ranking['names'].dtype.names

# Create an empty dictionary to store the top 15 genes for each group
top_genes_per_group = {}

# Iterate over each group to extract the top 15 genes
for group in groups:
    top_genes_per_group[group] = ranking['names'][group][:15]

# Convert the dictionary to a pandas DataFrame for easy viewing
top_genes_df = pd.DataFrame(top_genes_per_group)

top_genes_df.to_csv('top_genes_t_cells.csv')

In [16]:
sc.tl.umap(tcells)

In [ ]:
sc.pl.embedding(tcells, basis = 'umap', color = ['leiden', 'Xcr1', 'Clec9a', 'Xist', 'Cd8a', 'Cd4', 'Foxp3', 'Rbp2'], cmap='Blues', ncols=2)

In [25]:
t_cell_subcluster_annotations = {
    0: 'Segmentation Artifact Tumor Immune',
    1: 'Segmentation Artifact Tumor Immune',
    2: 'Segmentation Artifact Tumor Immune',
    3: 'Segmentation Artifact Tumor Immune',
    4: 'Segmentation Artifact Tumor Immune',
}


In [26]:
tcells.obs['cell_types'] = tcells.obs['leiden'].astype(int).map(t_cell_subcluster_annotations)

In [27]:
# Temporarily convert the column to a non-categorical type
adata.obs['cell_types'] = adata.obs['cell_types'].astype(str)

# Now assign the values
adata.obs.loc[adata.obs['cell_types'] == 'CD4 T Cells', 'cell_types'] = tcells.obs['cell_types'].astype(str)

# Optionally, convert back to categorical if needed
adata.obs['cell_types'] = adata.obs['cell_types'].astype('category')


In [ ]:
sc.pl.embedding(adata, basis = 'mde', color = ['batch', 'cell_types'])

The segmentation is not perfect, we might have to use resolveVI. Cell typing will need to be refined over an all day session. However, we will worry about that later and focus on our figure scripts. 

In [32]:
adata.write(os.path.join(input_file, "combined", 'cell_types_assigned.h5ad'))

In [ ]:
sc.pl.embedding(adata, basis = 'spatial', color = 'cluster_cellcharter')